In [1]:
import io
from IPython.nbformat import current
def execute_notebook(nbfile):
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    ip = get_ipython()
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)
execute_notebook("/cellar/users/ramarty/Projects/hla_ii/bin/imports.ipynb")
execute_notebook("/cellar/users/ramarty/Projects/hla_ii/bin/samples.ipynb")

Populating the interactive namespace from numpy and matplotlib
Populating the interactive namespace from numpy and matplotlib


/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/IPython/nbformat/current.py:19: UserWarning: IPython.nbformat.current is deprecated.

- use IPython.nbformat for read/write/validate public API
- use IPython.nbformat.vX directly to composing notebooks of a particular version

  """)


#### Establish available alleles

In [13]:
! /cellar/users/ramarty/programs/netMHCIIpan-3.1/netMHCIIpan -list > /cellar/users/ramarty/Data/hla_ii/presentation/other/netMHCIIpan_alleles.txt

### Gather from /nrnb

Work to develop script

In [2]:
lines = [x.split('\t') for x in open('/cellar/users/ramarty/Data/hla_ii/hla_types/result/sampleID_final.result.txt').readlines()[:8]]

In [6]:
patient_dictionary = {}
for line in lines:
    print line[0]
    if line[0] in ['A', 'B', 'C', 'DRB1']:
        if line[1] == 'Not typed':
            alleles = ['-', '-']
        elif line[2].strip() == '-':
            x = line[1]
            alleles = [line[0]+'_'+x.split('*')[1].split(':')[0]+x.split('*')[1].split(':')[1]]*2
        else:
            alleles = [line[0]+'_'+x.split('*')[1].split(':')[0]+x.split('*')[1].split(':')[1] for x in line[1:3]]
        patient_dictionary[line[0]+'_allele1'] = alleles[0]
        patient_dictionary[line[0]+'_allele2'] = alleles[1]
    # combinations
    else:
        print line
        if line[1] == 'Not typed':
            alleles = ['-', '-']
        elif line[2].strip() == '-':
            x = line[1]
            alleles = [line[0]+x.split('*')[1].split(':')[0]+x.split('*')[1].split(':')[1]]*2
        else:
            alleles = [line[0]+x.split('*')[1].split(':')[0]+x.split('*')[1].split(':')[1] for x in line[1:3]]
        patient_dictionary[line[0]+'_allele1'] = alleles[0]
        patient_dictionary[line[0]+'_allele2'] = alleles[1]

A
B
C
DRB1
DQA1
['DQA1', 'HLA-DQA1*05:01:01', 'HLA-DQA1*03:01:01\n']
DQB1
['DQB1', 'HLA-DQB1*03:01:01', 'HLA-DQB1*02:01:01\n']
DPA1
['DPA1', 'Not typed', 'Not typed\n']
DPB1
['DPB1', 'HLA-DPB1*03:01:01', 'HLA-DPB1*20:01:01\n']


In [46]:
patient_dictionary

{'A_allele1': 'A_1101',
 'A_allele2': 'A_3101',
 'B_allele1': 'B_5108',
 'B_allele2': 'B_1801',
 'C_allele1': 'C_1602',
 'C_allele2': 'C_0501',
 'DPA1_allele1': 'DPA10103',
 'DPA1_allele2': 'DPA10103',
 'DPB1_allele1': 'DPB10402',
 'DPB1_allele2': 'DPB10201',
 'DQA1_allele1': 'DQA10501',
 'DQA1_allele2': 'DQA10102',
 'DQB1_allele1': 'DQB10604',
 'DQB1_allele2': 'DQB10201',
 'DRB1_allele1': 'DRB1_1302',
 'DRB1_allele2': 'DRB1_0301'}

### Creating patient-allele dictionary

In [3]:
patient_types_df = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/hla_types/hla_types.tcga.csv', index_col=0)
patients = list(patient_types_df.index)

In [4]:
len(patient_types_df)

4725

In [29]:
patient_types_df.head()

,DPA1_allele1,DPA1_allele2,DPB1_allele1,DPB1_allele2,DQA1_allele1,DQA1_allele2,DQB1_allele1,DQB1_allele2,DRB1_allele1,DRB1_allele2
TCGA-04-1336,DPA10201,DPA10201,DPB11301,DPB10101,DQA10201,DQA10102,DQB10604,DQB10202,DRB1_1302,DRB1_0701
TCGA-04-1347,DPA10201,DPA10103,DPB10401,DPB11701\n,DQA10401,DQA10201\n,DQB10202,DQB10402,DRB1_0701,DRB1_0801
TCGA-04-1348,DPA10103,DPA10103,DPB10401,DPB10301,DQA10505,DQA10301,DQB10302,DQB10301,DRB1_1103,DRB1_0404
TCGA-04-1360,DPA10103,DPA10103,DPB10201,DPB11801\n,DQA10101,DQA10102,DQB10501,DQB10502,DRB1_1101,DRB1_0101
TCGA-04-1371,DPA10201,DPA10201,DPB11301,DPB10101,DQA10505,DQA10201\n,DQB10319,DQB10202,DRB1_0701,DRB1_1102


In [56]:
patient = 'TCGA-A3-3387'
DA = list(patient_types_df.ix[patient][['DPA1_allele1', 'DPA1_allele2']])
DB = list(patient_types_df.ix[patient][['DPB1_allele1', 'DPB1_allele2']])
print DA, DB
for a in DA:
    for b in DB:
        print 'HLA-{0}_{1}'.format(a, b)

['DPA10103', 'DPA10103'] [nan, nan]
HLA-DPA10103_nan
HLA-DPA10103_nan
HLA-DPA10103_nan
HLA-DPA10103_nan


In [46]:
list(patient_types_df.ix[patient][['DPA1_allele1']].isnull())[0]

False

In [57]:
dictionary = {}
for patient in patients:
    try:
        # DR
        alleles = list(patient_types_df.ix[patient][['DRB1_allele1', 'DRB1_allele2']])

        # DP 
        DA = list(patient_types_df.ix[patient][['DPA1_allele1', 'DPA1_allele2']])
        DB = list(patient_types_df.ix[patient][['DPB1_allele1', 'DPB1_allele2']])
        for a in DA:
            for b in DB:
                alleles.append('HLA-{0}_{1}'.format(a.strip(), b.strip()))

        # DQ
        DA = list(patient_types_df.ix[patient][['DQA1_allele1', 'DQA1_allele2']])
        DB = list(patient_types_df.ix[patient][['DQB1_allele1', 'DQB1_allele2']])
        for a in DA:
            for b in DB:
                alleles.append('HLA-{0}_{1}'.format(a.strip(), b.strip()))
        dictionary[patient] = alleles 
    except:
        # weeds out the patients with the failed typing
        print patient

pickle.dump(dictionary, open('/cellar/users/ramarty/Data/hla_ii/hla_types/TCGA.HLA_classII.p', 'w'))

TCGA-13-0805
TCGA-A2-A04U
TCGA-A2-A0T6
TCGA-A2-A0YC
TCGA-A2-A0YI
TCGA-A3-3387
TCGA-A5-A0R6
TCGA-A5-A0R7
TCGA-A5-A0VQ
TCGA-A7-A13E
TCGA-AA-3522
TCGA-AA-3529
TCGA-AA-A00Z
TCGA-AA-A01V
TCGA-AG-3600
TCGA-AG-3726
TCGA-AG-A020
TCGA-AG-A036
TCGA-AN-A0XS
TCGA-AO-A12D
TCGA-AR-A0U3
TCGA-AR-A1AU
TCGA-AX-A06J
TCGA-B0-5695
TCGA-B6-A0RH
TCGA-B6-A0RQ
TCGA-BG-A0RY
TCGA-BH-A0B7
TCGA-BH-A0BC
TCGA-BH-A0BO
TCGA-BH-A0EI
TCGA-BH-A18I
TCGA-BH-A18Q
TCGA-BS-A0UL
TCGA-BS-A0UT
TCGA-C8-A12Z
TCGA-C8-A131
TCGA-CZ-5459
TCGA-CZ-5465
TCGA-D1-A16V
TCGA-D1-A16Y
TCGA-D1-A175
TCGA-D1-A176
TCGA-D1-A177
TCGA-D1-A17B
TCGA-D1-A17C
TCGA-D1-A17D
TCGA-D1-A17F
TCGA-D1-A17K
TCGA-D8-A143
TCGA-E2-A10A
TCGA-E2-A14T
TCGA-E2-A156
TCGA-E2-A15C
TCGA-E2-A15L
TCGA-E2-A15O
TCGA-E2-A15R
TCGA-E2-A1B0
TCGA-E2-A1B1


In [58]:
len(dictionary.keys())

1841

### Debugging

In [2]:
df = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/hla_types/hla_types.tcga.csv', index_col=0)

In [3]:
df = df.replace('-', np.nan)

In [4]:
df.columns

Index([u'DPA1_allele1', u'DPA1_allele2', u'DPB1_allele1', u'DPB1_allele2',
       u'DQA1_allele1', u'DQA1_allele2', u'DQB1_allele1', u'DQB1_allele2',
       u'DRB1_allele1', u'DRB1_allele2'],
      dtype='object')

In [5]:
len(df.dropna())

7926

In [6]:
len(df[['DRB1_allele1', 'DRB1_allele2']].dropna())

8257